# Load in Data and Required Packages

In [1]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
from dash.dependencies import Input, Output, State
import inspect
import pandas as pd
import numpy as np
import warnings
import dash_bootstrap_components as dbc
import dash_table_experiments as dt
from dash.exceptions import PreventUpdate
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import json
import portalocker
import dash_utils

<ipython-input-1-e1c025d9a8af>:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-e1c025d9a8af>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-e1c025d9a8af>:5: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


In [2]:
import json

data_path = 'questions.json'

# Open and read the JSON file
with open(data_path, 'r') as json_file:
    data = json.load(json_file)
# Create a DataFrame from the JSON data
df = pd.DataFrame(data)
df

,question,solution,answer
0,"The Culinary Institute of America, the best cu...",def solution():\n #40 people paid $3000 for...,720000.000000
1,Melanie has $45. She shops at a grocery store ...,def solution():\n #Melanie started with $45...,0.800000
2,Dianne is Dunkin' Brands CFO. She has to cut 5...,def solution():\n #Cut 5% of the budget for...,4.940000
3,"The scone cost $4, the muffin cost $3, the don...","def solution():\n #The scone cost $4, the m...",24.000000
4,4 pizzas cost $32. 2 pizzas cost $16. How much...,def solution():\n # 4 pizzas cost $32\n ...,8.000000
5,Gina is in a candy store that sells 10 gumball...,def solution():\n #Gina has 5 dollars\n ...,0.500000
6,"For my 18th birthday, my parents gave me $100....",def solution():\n #I got $100 for my 18th b...,1459.374246
7,Chase and his parents are going on a trip to t...,def solution():\n #Chase and his parents ar...,275.000000


## Load dataframe for storing data

In [3]:
csv_filename = 'feedback.csv'
feedback = pd.read_csv(csv_filename)
feedback = feedback[feedback.question_type != 'test']
feedback

,question1,question2,selected_question,grade_level,math_operations,question_type


# Build Dash App

In [4]:
markdown_text = '''
Please read the following two questions carefully. Select the question you think is better. Make a decision based on:
\na) the correctness of the solution (you may use a calculator or internet to verify the solution) 
\nb) the correctness of the reasoning contained within the Python code that executes the solution
\nc) the appropriateness of the question (i.e., is it a strange question?, is it appropriate for a K-12 student?)


\nIf both questions are equally good based on the criteria above, you may select the 'Tie- both good' option. If both questions are equally bad based on the criteria above, you may select the 'Tie- both bad' option. Please use these options sparingly and try to declare a clear winner whenever possible. If the question/reasoning is nonsensical or highly ungrammatical/confusing, label it with "F" (for fail) rather than selecting a winnner.


\nAfter selecting the question you think is best, please answer the follow-up questions regarding the grade level, mathematical operations present in the question, and general topic the question is about. For labelling grade level, please see this [guide](https://braintrusttutors.com/math-skills-by-grade-level/) for more directions on the key topics covered in each grade level (note: you will need to right click the link and open the link in a new tab to avoid leaving this webpage).
'''

In [5]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [6]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        html.H1("Math Question Generator Feedback Portal"),    
        
        dcc.Markdown(children = markdown_text, style = {'white-space': 'pre-wrap'}, dangerously_allow_html=True),
        
        html.Div([
            
            html.Button(id='submit', n_clicks=0, children = 'Display Questions', style = {'fontsize': '12'}),
            dcc.Store(id='button-clicked', data=0)

        ], style={'width': '12%', 'float': 'left', 'white-space': 'pre-wrap'}),
        
        html.Div([
            html.H2(id = 'question1_label'),
            dcc.Markdown(id='question1'),
            dcc.Markdown(id= 'question1_solution'),
            dcc.Markdown(id = 'question1_answer'),
        ], id = 'question1_box', style={'width': '40%', 'float': 'left', 'font-weight': 'bold', 'font-size': '12px', 'white-space': 'pre-wrap', 'display': 'inline-block'}), 
        
        html.Div([
            html.H2(id = 'question2_label'),
            dcc.Markdown(id='question2'),
            dcc.Markdown(id= 'question2_solution'),
            dcc.Markdown(id = 'question2_answer'),
        ], id = 'question2_box', style={'width': '40%', 'float': 'right', 'justify-content': 'space-between', 'font-weight': 'bold', 'font-size': '12px', 'display': 'inline-block', 'white-space': 'pre-wrap'}),
        
        html.Div(
            [
                html.Label('\n Select the best option:', style={'font-weight': 'bold'}),
                dcc.RadioItems(
                    id='selected_question',
                    options=[
                        {'label': 'Question 1', 'value': 'Question 1'},
                        {'label': 'Question 2', 'value': 'Question 2'},
                        {'label': 'Tie- both good', 'value': 'Tie- both good'},
                        {'label': 'Tie- both bad', 'value': 'Tie- both bad'},
                        {'label': 'Question 1 F', 'value': 'Question 1 F'},
                        {'label': 'Question 2 F', 'value': 'Question 2 F'},
                    ]
                ),
                
                html.Label('If applicable, select the grade level the winning question is appropriate for:', style = {'font-weight': 'bold'}), 
                dcc.RadioItems(id='grade_level',
                options=['K', 1, 2, 3, 4, 5, 6, 7, 8, 'High School']),
                
                html.Label('If applicable, select the primary mathematical operation(s) present in the winning question:', style = {'font-weight': 'bold'}), 
                dcc.Checklist(id = 'math_operations', options = ['Addition', 'Subtraction', 'Division', 'Multiplication', 'Fractions', 'Decimals']),
                
                html.Label('If applicable, what was the winning question about (i.e., soccer, money, food, shopping, etc.)? Please enter one word or phrase only.', style = {'font-weight': 'bold'}),
                dcc.Input(id = 'question_type', type='text', placeholder = "Enter your answer here.", debounce = True),
                
                html.Div(html.Button('Submit Feedback', id='submit-val', n_clicks=0)),
                
                dcc.Markdown(id = 'feedback')
            ],
            style={'width': '100%', 'display': 'none', 'whitespace': 'pre-wrap', 'position': 'absolute', 'left': 0},
            id='standard-container'), 
        
        html.Div([
              dcc.Location(id='url', refresh=False),
              html.Div(id='page-content')
])
    ]
)

@app.callback([Output(component_id="question1",component_property="children"), 
               Output(component_id="question1_solution",component_property="children"), 
              Output(component_id = 'question1_answer', component_property = 'children'), 
               Output(component_id = 'question1_label', component_property = 'children'),
              Output(component_id= "question2", component_property="children"), 
               Output(component_id="question2_solution",component_property="children"), 
              Output(component_id = 'question2_answer', component_property = 'children'),
              Output(component_id = 'question2_label', component_property = 'children')],
                  [Input(component_id = 'submit', component_property= 'n_clicks')])

def pull_questions(n_clicks):
    global df 
    if n_clicks>0:
        df_temp = df.sample(n = 2, replace= False)
        question1 = df_temp.iloc[0]['question']
        question1 = "Question: \n" + question1
        question1_solution = df_temp.iloc[0]['solution']
        question1_solution = "Solution: \n" + question1_solution
        question1_answer = "Answer: \n" + str(df_temp.iloc[0]['answer'])
        question1_label = 'Question 1'
        
        question2 = df_temp.iloc[1]['question']
        question2 = "Question: \n" + question2
        question2_solution = df_temp.iloc[1]['solution']
        question2_solution = "Solution: \n" + question2_solution
        question2_answer = "Answer: \n" + str(df_temp.iloc[1]['answer'])
        question2_label = "Question 2"
        return question1, question1_solution, question1_answer, question1_label, question2, question2_solution, question2_answer, question2_label
        
    if n_clicks==0:
        return '', '', '', '', '', '', '', ''

@app.callback(
    Output('standard-container', 'style'),
    [Input('submit', 'n_clicks')],
    State('button-clicked', 'data')
)
def show_hide_radioitems(n_clicks, button_clicked):
    if n_clicks > button_clicked:
        return {'display': 'block', 'float': 'left', 'width': '50%'}
    else:
        return {'display': 'none'}

@app.callback([Output('feedback', 'children'),
              Output('submit-val', 'n_clicks')],
              [Input('submit-val', 'n_clicks'),
              Input('selected_question', 'value'),
              Input('grade_level', 'value'),
              Input('math_operations', 'value'),
              Input('question_type', 'value'),
              Input('question1', 'children'),
              Input('question1_solution', 'children'), 
              Input('question2', 'children'), 
              Input('question2_solution', 'children')])

def export_feedback(n_clicks, selected_question, grade_level, math_operations, question_type, question1, question1_solution, question2, question2_solution):
    if n_clicks > 0:
        global feedback
        global df
        feedback = feedback.append({'question1': question1 + question1_solution, 
                                                  'question2': question2 + " " + question2_solution,
                                                'selected_question': selected_question,
                                                  'grade_level': grade_level,
                                                  'math_operations': math_operations,
                                                  'question_type': question_type}, ignore_index=True)
        #Discuss if I want to keep this logic here:
        if selected_question == 'Question 1 F':
            question1 = question1.split('Question: \n')[1].strip()
            df = df[df.question != question1]
            selected_question = ''
            
        if selected_question == 'Question 2 F':
            question2 = question2.split('Question: \n')[1].strip()
            df = df[df.question != question2]
            selected_question = ''
        
        if selected_question == "Tie- both bad":
            question1 = question1.split('Question: \n')[1].strip()
            df = df[df.question != question1]
            question2 = question2.split('Question: \n')[1].strip()
            df = df[df.question != question2]
            selected_question = ''
            
        return 'Feedback submitted successfully.', 0
    else:
        return '', 0

@app.callback(Output('url', 'refresh'),
              [Input('submit-val', 'n_clicks')])  

def reset(n_clicks):
  if n_clicks > 0:
    return True # refresh page

if __name__== "__main__":
    app.run_server(mode= 'external', host = "0.0.0.0", debug=True)

Dash app running on http://0.0.0.0:8050/


## Save feedback

In [7]:
feedback.to_csv(csv_filename, index=False, header = True) 

# Format feedback for reward model

In [18]:
chosen = []
rejected = []

for i in range(0, len(feedback)):
    if feedback['selected_question'].iloc[i] == "Question 1":
        chosen.append(feedback['question1'].iloc[i])
        rejected.append(feedback['question2'].iloc[i])
        
    if feedback['selected_question'].iloc[i] == "Question 2":
        chosen.append(feedback['question2'].iloc[i])
        rejected.append(feedback['question1'].iloc[i])
    
    if feedback['selected_question'].iloc[i] == "Question 2 F":
        chosen.append(feedback['question1'].iloc[i])
        rejected.append(feedback['question2'].iloc[i])
    
    if feedback['selected_question'].iloc[i] == "Question 1 F":
        chosen.append(feedback['question2'].iloc[i])
        rejected.append(feedback['question1'].iloc[i])
        
reward_df = pd.DataFrame({'chosen': chosen, 'rejected': rejected})
csv_filename = 'reward.csv'
reward_df.to_csv(csv_filename, index=False, header = True) 